<a href="https://colab.research.google.com/github/yeijSong/HandsOnDataAnalysis/blob/main/06_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EC%9B%94%EA%B0%84_TOP100_%ED%81%AC%EB%A1%A4%EB%A7%81%26%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 네이버 영화 월간 TOP 100 데이터 크롤링하기

라이브러리 설치

In [1]:
# 링크 수집을 위한 라이브러리 설치
!pip install selenium

     |████████████████████████████████| 911kB 12.0MB/s 


라이브러리 임포트

In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd 

In [3]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.download.n

**분석할 데이터 모으기(크롤링)**

네이버 영화 TOP100 페이지별 링크 수집

In [4]:
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page=(1,5) 실시간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page=(1,5) 일간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page=(1,5) 주간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page=(1,5) 월간

def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = [] # 100개의 영화링크를 수집할 리스트 변수 선언
  
  for pageNo in range(startPageNo-1, lastPageNo) :
    # 실시간 TOP 100 영화 링크 수집
    url = 'https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page=' + str(pageNo+1)

    req = requests.get(url) # 페이지 접속
    soup = BeautifulSoup(req.text, 'lxml')
    # html 파싱(원하는 위치에 있는 값을 찾아줌)을 위한 객체 생성, 불러올 데이터 정보가 lxml형식이다
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]')
    # 해당페이지의 페이지소스 중 378line 확인
    # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">
    # div 일종의 박스, 여기 안에 무슨 정보를 넣을 거다
    # ul(반복되는 정보) 안에 li가 들어있다(주로 그렇기 때문에 세트로 생각하면 좋다)

    # movielinks의 링크는 완전한 형태의 링크가 아니기 때문에 부족한 앞부분을 채워 완전한 링크를 만들어줘야함

    for movielink in movielinks :
      link = str(movielink.get('href'))

      links.append("https://series.naver.com" + link) # 접속가능한 전체링크 형태로 변환하여 links 리스트에 넣어줌
      
  return links


네이버 실시간 TOP100 영화 제목, 평점, 장르, 줄거리 크롤링

In [5]:
def getMovieDataFromNaverSeries(links):
  title_infos = [] # 제목
  content_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  url2 = "https://www.naver.com"
  # 원하는 링크로 지속적으로 바로 들어가면 해당 사이트에서 비정상적인 접근으로 인식하여 내 IP차단할 가능성 있음
  
  driver = webdriver.Chrome('chromedriver', options=options) # 나 대신 링크에 들어갈 드라이버 생성
  driver.get(url2)
  time.sleep(3.0) # 3초 정도 sleep(사람이 진짜 보는 것처럼 인식하도록)

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't')
  # 새 창을 열어서 해당 링크로 가게끔하여 사람처럼 느끼도록

  for link in links:
    print(link + '수집중......')

    driver.switch_to.window(driver.window_handles[-1]) 
    
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0])
    # 새창 활성화(새 탭이 열린 것을 클릭하여 들어갈 수 있도록)
    time.sleep(3.0)

    html_source = driver.page_source # req.text와 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년 관람불가 영화 크롤링시, 연령인증위한 로그인을 하라고 하니 수집에서 제외시킴
    flag = html_soup.text[0:10] # 이 부분을 확인하면 로그인 화면인지 알 수 있음
    
    newflag= ''.join(flag)
    newflag= newflag.replace('\n','')

    if newflag == '네이버' :
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em')
      score = float(score.text)
      score = int(score)
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent') # 정보들 중에 텍스트로 된 것만 가져와 (장르, 장르명) 두 가지가 들어올 예정임
      genre = genre.replace('장르','')
      genre = genre.replace('\n','')
      genre = genre.replace('\t','')
      genre_infos.append(genre)

      # 제목, 줄거리, 개봉일
      
      try :
        movieinfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      except :
        movieinfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')

      # 영와 상세보기 페이지로 이동
      movie_req = requests.get(movieinfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find('meta',{'property':'og:title'}).get('content')
      title_infos.append(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0 :
        content_infos.append('줄거리 오류')

      else :
        for contents in contents_texts:
          # 줄거리 데이터 클렌징 작업
          temp = contents.text
          temp = temp.replace('\r','') # 줄바꿈 제거
          temp = temp.replace('\xa0','') # 공백 제거
          content_infos.append(temp)

    elif newflag == '' :
      print('청소년 관람불가 영화로 데이터를 수집하지 않습니다.')
  
  print('수집을 완료합니다.')
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()
  
  # 수집한 영화정보들을 하나의 파일로 만들기 위해 딕셔너리에 넣어줌
  movie_dic = {'제목' : title_infos, '평점' : score_infos, '장르' : genre_infos, '줄거리' : content_infos}
  
  # 딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)
  
  # 수집된 정보 중에 중복 데이터 삭제(줄거리를 비교하여 첫번째 수집 영화데이터만 남기고 삭제)
  movie_df2 = movie_df.drop_duplicates('줄거리', keep='first')

  return movie_df2

CSV파일로 저장

In [6]:
def dftoCsv(movie_df, num):
  try:
    movie_df.to_csv(('movie_my_date_'+str(num)+'.csv'),
                    sep=',',
                    na_rep='NaN',
                    encoding='euc-kr')
  except:
    print('Error')

크롤링 실행

In [7]:
#1) 크롤링 할 링크 수집

links = getPageLinksWantRange(1,5)

#2) 영화 상세정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

#3) csv파일 저장
dftoCsv(movie_df2, len(movie_df2))


https://series.naver.com/movie/detail.nhn?productNo=6119777수집중......
7
https://series.naver.com/movie/detail.nhn?productNo=6176165수집중......
8
https://series.naver.com/movie/detail.nhn?productNo=6158879수집중......
9
https://series.naver.com/movie/detail.nhn?productNo=6098871수집중......
9
https://series.naver.com/movie/detail.nhn?productNo=6023769수집중......
9
https://series.naver.com/movie/detail.nhn?productNo=6139377수집중......
8
https://series.naver.com/movie/detail.nhn?productNo=6139375수집중......
9
https://series.naver.com/movie/detail.nhn?productNo=6119778수집중......
7
https://series.naver.com/movie/detail.nhn?productNo=4356726수집중......
9
https://series.naver.com/movie/detail.nhn?productNo=6176518수집중......
청소년 관람불가 영화로 데이터를 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=3293948수집중......
청소년 관람불가 영화로 데이터를 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=6039515수집중......
0
https://series.naver.com/movie/detail.nhn?productNo=6176542수집중......
8
https://series.naver.co